In [46]:
import requests
from bs4 import BeautifulSoup
import string
import urlparse 
import re
import locale
import pickle
from dateutil import parser

In [39]:
def get_details(url, movie):
    print movie 
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page) 
    
    try:
        rt_score = soup.find(id="movie_ratings").find_all('a')[2].br.next_sibling
        rt_score = str(rt_score)
        match = re.search(r'\d+', rt_score)
        critic_score = int(match.group())
    except:
        critic_score = ""
    
    rows = soup.find_all('table')[2].find_all('tr')
    for row in rows:
        if row.find_all('td')[0].text == u'Production\xa0Budget:':
            budget = row.find_all('td')[1].text
            locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )
            prod_budget = locale.atoi(budget.strip("$"))
        if row.find_all('td')[0].text =='Domestic Releases:':
            wide = '(Wide)' in row.find_all('td')[1].text
            release =  row.find_all('td')[1]
            release = str(release.text).split(" (")
            release_dt = parser.parse(release[0])
            company = str(row.find_all('td')[1].find('a').text)
        if row.find_all('td')[0].text == u'MPAA\xa0Rating:':
            rating = str(row.find_all('td')[1].find('a').text)
        if row.find_all('td')[0].text == u'Production\xa0Method:':
            prod_method = str(row.find_all('td')[1].find('a').text)
        if row.find_all('td')[0].text =='Running Time:':
            runtime = row.find_all('td')[1].text
            runtime = runtime.replace(' minutes','')
            run_time = int(runtime)
        if row.find_all('td')[0].text =='Source:':
            source = str(row.find_all('td')[1].find('a').text)
        if row.find_all('td')[0].text =='Genre:':
            genre = str(row.find_all('td')[1].find('a').text)

    try:
        movies_dict[movie]['rating']= rating
    except: 
        movies_dict[movie]['rating']= ""
    try:
        movies_dict[movie]['budget']= prod_budget
    except: 
        movies_dict[movie]['budget']= ""
    try:
        movies_dict[movie]['budget']= prod_budget
    except:
        movies_dict[movie]['budget']= ""
    try:
        movies_dict[movie]['method']= prod_method
    except:
        movies_dict[movie]['method']= ""
    try:
        movies_dict[movie]['source']= source
    except:
        movies_dict[movie]['source']= ""
    try:    
        movies_dict[movie]['genre']= genre
    except:
        movies_dict[movie]['genre']= ""
    try:
        movies_dict[movie]['distribution']= company
    except:
        movies_dict[movie]['distribution']= ""
    try:
        movies_dict[movie]['runtime']= run_time
    except: 
        movies_dict[movie]['runtime']= ""
    try:
        movies_dict[movie]['critic_rating']= critic_score
    except:
        movies_dict[movie]['critic_rating']= ""
   

In [40]:
def get_weekly(url, movie):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page) 
    rows = soup.find_all(id = 'box_office_chart')[-1].find_all('tr')
    weekly_list = []
    for row in rows[1:]:
        row_list = []
        col = row.text
        for cell in row.find_all('td'):
            info = filter(lambda x: x in string.printable, cell.text)
            info = str(info)
            row_list.append(info)
        weekly_list.append(row_list)

    movies_dict[movie]['weekend_boxoffice']= weekly_list

In [12]:
def get_movieid(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)
    rows = soup.find('table').find_all('tr')[1:201]
    for row in rows:
        try:
            title = filter(lambda x: x in string.printable, row.a.text)
            title = str(title)
            movies_dict[title]= {}
            movie_id = row.a['href']
            par = urlparse.urlparse(movie_id).path.lstrip("/movie")
            movies_dict[title]['id']= par
        except:
            print row

In [7]:
def create_dict(year):
	url = 'http://www.the-numbers.com/market/%d/top-grossing-movies' % (year)
	while year < 2015:
		get_movieid(url)
		year += 1

In [37]:
def scrape(movies_dict):
    for movie in movies_dict: 
        summary_url = 'http://www.the-numbers.com/movie/%s#tab=summary' % (str(movies_dict[movie]['id']))
        try:
            get_details(summary_url, movie)
        except:
            print "------Could not get details for " + movie
        boxoffice_url = 'http://www.the-numbers.com/movie/%s#tab=box-office' % (str(movies_dict[movie]['id']))
        try:
            get_weekly(boxoffice_url, movie)
        except: 
            print "-----Could not get weekly for " + movie
        pickle_count += 1

In [41]:
movies_dict= {}


In [43]:
create_dict(2003)


In [44]:
scrape(movies_dict)

Gone Girl
Endless Love
Muppets Most Wanted
Thor: The Dark World
The Fluffy Movie
When the Game Stands Tall
Dallas Buyers Club
The Best of Me
Bad Words
The Theory of Everything
Paranormal Activity: The Marked Ones
The Lunchbox
Kill the Messenger
Birdman or (The Unexpected Virtue of Ignorance)
The Monuments Men
Divergent
A Walk Among the Tombstones
Big Hero 6
Transformers: Age of Extinction
Her
The Raid 2
Exodus: Gods and Kings
Lone Survivor
The Good Lie
The Grand Seduction
Get on Up
A Haunted House 2
Saving Mr. Banks
Under the Skin
Veronica Mars
Island of Lemurs: Madagascar
American Hustle
The Book of Life
Begin Again
The Hundred-Foot Journey
Labor Day
Legends of Oz: Dorothys Return
The Wolf of Wall Street
Grudge Match
Meet the Mormons
12 Years a Slave
The Hobbit: The Battle of the Five Armies
Dolphin Tale 2
This is Where I Leave You
The Hobbit: The Desolation of Smaug
Noah
Beyond the Lights
Cantinflas
Tyler Perry's The Single Moms Club
The Drop
The Hunger Games: Catching Fire
Hercules


In [ ]:
with open('list_of_movie_info_dicts.pkl', 'w') as picklefile:
    pickle.dump(movies_dict, picklefile)
